### Assignment 3
_Jiajie Lu_

Consider the equipment location example; its text is repeated for convenience. A service man moves between 4 sites, with site 1 denoting home oﬃce, and 2, 3, and 4 denoting remote sites. Work at sites 2,3, and 4 requires the use of an equipment trailer. The cost of relocating the equipment trailer between the sites is $d(j,k) = 300$ for $k\ne j$. The cost $c(k,j)$ of using the trailer is $100$ if the work is at site $k>1$ and trailer is at site $j\ne k$ with $j > 1$; $50$ if $j = k$ and $j>1$, and $200$ if the work is at $k>1$ and the trailer is at $j=1$ (home oﬃce). 
$$
c(k,j)=\left\{
\begin{aligned}
100 && k>1,j\ne k\\
50 && k>1,j= k\\
200 && k>1,j=1
\end{aligned}
\right.
$$
If the service man is at the home oﬃce, no work is done and cost is zero. At any time, the service man knows their location, observes the location of the next job (or 1 if no job is to be done) and decides whether and where to move the trailer. The transition matrix between job locations is
$$
p=\begin{bmatrix}
0.1 & 0.3 & 0.3 & 0.3\\ 
0.0 & 0.5 & 0.5 & 0.0\\ 
0.0 & 0.0 & 0.8 & 0.2\\ 
0.4 & 0.0 & 0.0 & 0.6
\end{bmatrix}
$$

For example, if the current location (job) is 2, the probability that the next job is at 3 is 0.5. Assume the discount factor of 0.95.

(a)  Formulate a Markov decision problem to help the repairment decide on the movement of the trailer: deﬁne the state and control space and write the dynamic programming equations.

>* State Space :
$$\mathcal{X}=\{x_t=(j_t,k_t)\in\{1,2,3,4\}\times\{1,2,3,4\}\}$$ 
where $j_t$ is current trailer position and $k_t$ is current job position.
* Control Space : $$\mathcal{U}=\{1,2,3,4\}$$ is the next trailer position.
* Transition kernel:
$$
P[(j_{t+1},k_{t+1})|j_t,k_t,u_t]=\left\{
\begin{aligned}
p_{k_t,k_{t+1}} && j_{t+1}=u_t\\
0 && \text{otherwise}
\end{aligned}
\right.
$$
* Step-wise Cost function conditioned with control (j-location of the equipment; k-location of the next job):
$$
c(j,k,u)=\left\{
\begin{aligned}
0 && u=j, k=1\\
50 && u=j=k>1\\
100 && u=j, k\ne j, j\ne1,k>1\\
200 && u=j=1, k>1, \\
300 && u\ne j, k=1\\
300+50 && u\ne j, u= k, k>1\\ 
300+100 && u\ne j, u\ne k,u\ne1, k>1\\
300+200 && u\ne j, u=1, k>1
\end{aligned}
\right.
$$
* Dynamic programming equation
$$
v(j,k)=\min_{u\in\{1,2,3,4\}}\{c(j,k,u)+0.95\sum_{l=1}^{4}v(u,l)p_{kl}\}
$$

In [1]:
import numpy as np

# Transition Probability Matrix
# For convenience we set up the 5x5 
# so that ew can obtain P[j,k] = p_{jk}
P = np.array([
    [.0, .0, .0, .0, .0],
    [.0, .1, .3, .3, .3],
    [.0, .0, .5, .5, .0],
    [.0, .0, .0, .8, .2],
    [.0, .4, .0, .0, .6]
])

# control space
U = np.array([1,2,3,4])

# step-wise cost function
# c[j,k,u]
# j-location of equipment,
# k-location of next job,
# u-control
C = np.zeros((5,5,5))

# discounted factor
alpha = 0.95

# function to get value
def get_value(j, k, u):
    if u == j:
        if k == 1: return 0
        if k == j: return 50
        if j == 1: return 200
        
        return 100
    else:
        if k == 1: return 300
        if u == k: return 350
        if u == 1: return 500
        
        return 400

# update cost function
for i in U:
    for j in U:
        for u in U:
            C[i,j,u] = get_value(i, j, u)

(b)  Solve these equations by the value iteration method (starting from 0) and calculate lower and upper bound at each step.

In [2]:
# Value Iteration
def value_iteration(T, max_iter=1000, tol=0.0001):
    # TxT is the length of state space
    # initial value function
    v = np.zeros((T+1, T+1))
    v_old = np.zeros((T+1, T+1)) + 10000
    # inital policy path
    p = np.zeros((T+1, T+1))
    # current iteration
    current_step = 0
    
    # value iteration method
    while (np.linalg.norm(v - v_old) > tol) and (current_step < max_iter):
        v_old = v.copy()
        current_step += 1
        
        for j in U:
            for k in U:
                tmp = np.array([C[j, k, u] + 0.95*np.sum([P[k, l]*v_old[u, l] for l in U]) for u in U])
                v[j, k] = np.min(tmp)
                p[j, k] = np.argmin(tmp) + 1
                
        upper, lower = np.max(v - v_old), np.min(v - v_old)
        
    return v[1:, 1:], p[1:, 1:], upper, lower, current_step 
    

v, p, upper, lower, iters = value_iteration(4)
print(f"With {iters} iterations:")
print("We got the optimal v star")
print(v)
print("We got the optimal policy")
print(p)
print("With upper bound")
print(v + alpha/(1-alpha)*upper)
print("And lower bound")
print(v + alpha/(1-alpha)*lower)

With 290 iterations:
We got the optimal v star
[[1497.76357116 1618.05362423 1546.26984154 1591.73286596]
 [1428.21779544 1494.24410042 1546.26984154 1494.70404711]
 [1220.43021183 1318.05362423 1246.26984154 1311.07781043]
 [1330.1188323  1492.68947665 1439.28841927 1291.73286596]]
We got the optimal policy
[[1. 3. 3. 4.]
 [2. 2. 3. 2.]
 [3. 3. 3. 3.]
 [4. 4. 4. 4.]]
With upper bound
[[1497.76404934 1618.05410241 1546.27031972 1591.73334414]
 [1428.21827362 1494.2445786  1546.27031972 1494.70452528]
 [1220.43069001 1318.05410241 1246.27031972 1311.0782886 ]
 [1330.11931047 1492.68995483 1439.28889744 1291.73334414]]
And lower bound
[[1497.76357116 1618.05362423 1546.26984154 1591.73286596]
 [1428.21779544 1494.24410042 1546.26984154 1494.70404711]
 [1220.43021183 1318.05362423 1246.26984154 1311.07781043]
 [1330.1188323  1492.68947665 1439.28841927 1291.73286596]]


(c)  Solve the problem by the policy iteration method starting from “do not move trailer from base” at each state.

In [3]:
# Policy iteration
def policy_iteration(T, max_iter=1000, tol=0.001):
    # inital policy matrix
    p = np.array([0 for _ in range(T**2)])
    p_old = np.array([0 for _ in range(T**2)]) + 10
    # current step
    current_step = 0
    
    # policy iteration method
    while (np.linalg.norm(p - p_old) > tol) and (current_step < max_iter):
        p_old = p.copy()
        current_step += 1
        
        # init coef
        A = np.zeros((T**2, T**2))
        # init bias
        b = np.array([0 for _ in range(T**2)])
        # set up coef and bias
        for row, j in enumerate(U):
            for col, k in enumerate(U):
                idx = row*T + col
                b[idx] = -C[j, k, p[idx]]
                policy = p_old[idx]
                
                for l in range(T):
                    A[idx, (p[idx]-1)*T+l] += alpha*P[k, l+1]
        
        # set up A    
        A = A - np.eye(T**2)
        # solve the linear equation Ax=b
        v = np.linalg.solve(A, b)
        # update policy
        for row, j in enumerate(U):
            for col, k in enumerate(U): 
                tmp = np.array([C[j, k, u] + 0.95*np.sum([P[k, l]*v[(u-1)*T+l-1] for l in U]) for u in U])
                p[row*T + col] = np.argmin(tmp) + 1
                
    return v.reshape((T,T)), p.reshape((T,T)), current_step, b

v, p, iters, b = policy_iteration(4)
print(f"With {iters} iterations:")
print("We got the optimal v star")
print(v)
print("We got the optimal policy")
print(p)

With 4 iterations:
We got the optimal v star
[[1497.76402404 1618.05406446 1546.27028177 1591.73334414]
 [1428.21823567 1494.24454065 1546.27028177 1494.70448733]
 [1220.43065206 1318.05406446 1246.27028177 1311.07825065]
 [1330.11931047 1492.68995483 1439.28889744 1291.73334414]]
We got the optimal policy
[[1 3 3 4]
 [2 2 3 2]
 [3 3 3 3]
 [4 4 4 4]]


(d) Solve the problem by linear programming.

$$
\begin{aligned}
\max & \sum_{j,k} v(j, k)\\
\text{s.t.} & v(j, k)\le c(j,k,u)+\alpha \sum_{l\in\mathcal{X}}\left[ v(u, l)p_{kl} \right],\quad (j,k)\in\mathcal{X}
\end{aligned}
$$

In [4]:
from scipy.optimize import linprog

# linear programming
def linear_programming(T):
    # min c^Tv
    # Av <= b
    # parameter array c
    c = np.array([-1.0 for _ in range(T**2)])
    # coef matrix
    A = np.zeros((T**2*len(U), T**2))
    # const 
    b = np.array([0.0 for _ in range(T**2*len(U))])
    
    for row, j in enumerate(U):
        for col, k in enumerate(U):
            for i, u in enumerate(U):
                # current index
                r_idx = row*(T**2) + col*T + i
                c_idx = row*T + col
                # set entry value
                A[r_idx, c_idx] += 1.0
                b[r_idx] = C[j, k, u]
                
                for l in range(T):
                    A[r_idx, i*T+l] -= alpha*P[k, l+1]
                    
    # solve linear programming
    v = linprog(c, A, b)
    
    return v['x'].reshape(T,T)

v = linear_programming(4)
print("V star by dynamic programming:")
print(v)

V star by dynamic programming:
[[1497.76402382 1618.05406424 1546.27028155 1591.73334389]
 [1428.21823543 1494.24454041 1546.27028157 1494.70448709]
 [1220.4306519  1318.05406429 1246.27028161 1311.07825049]
 [1330.1193103  1492.68995463 1439.28889726 1291.73334397]]
